# 실습 - 🏀농구선수의 게임 기록 학습 후 포지션 예측

## 최적의 SVM 파라미터 찾기

### RBF 커널 - 2개의 파라미터
1. 비용(C) : 마진의 너비 조정
2. 감마 : 결정 경계선의 곡률을 조정

### 사이킷런 - 그리드 서치(gridsearch) : 간편하게 최적 비용, 감마 알 수 있음

In [20]:
import pandas as pd

df = pd.read_csv('data/csv/basketball_stat.csv')
df

,Player,Pos,3P,2P,TRB,AST,STL,BLK
0,Alex Abrines,SG,1.4,0.6,1.3,0.6,0.5,0.1
1,Steven Adams,C,0.0,4.7,7.7,1.1,1.1,1.0
2,Alexis Ajinca,C,0.0,2.3,4.5,0.3,0.5,0.6
3,Chris Andersen,C,0.0,0.8,2.6,0.4,0.4,0.6
4,Will Barton,SG,1.5,3.5,4.3,3.4,0.8,0.5
...,...,...,...,...,...,...,...,...
95,Nikola Vucevic,C,0.3,6.1,10.4,2.8,1.0,1.0
96,Dwyane Wade,SG,0.8,6.2,4.5,3.8,1.4,0.7
97,Dion Waiters,SG,1.8,4.3,3.3,4.3,0.9,0.4
98,Hassan Whiteside,C,0.0,7.0,14.1,0.7,0.7,2.1


In [22]:
# 사이킷런의 train_test_split로 데이터 나눔
from sklearn.model_selection import train_test_split

In [23]:
# 데이터에서 20%를 테스트 데이터로 분류함
train, test = train_test_split(df, test_size=0.2)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
import numpy as np

In [25]:
def svc_param_selection(X, y, nfolds):
    svm_parameters =[
        {'kernel': ['rbf'],
         'gamma' : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
         'C' : [0.01,0.1,1,10,100,1000]        
        }]
    # 사이킷런에서 제공하는 GridSearchCV를 사용해 최적의 파라미터를 구함
    clf = GridSearchCV(SVC(), svm_parameters, cv=10)
    clf.fit(X_train, y_train.values.ravel())
    print(clf.best_params_)
    
    return clf

In [26]:
# 3점슛과 블로킹 횟수를 학습 데이터로 사용
X_train = train[['3P', 'BLK']]

#농구 선수 포지션을 예측값으로 선정
y_train= train[['Pos']]

# 최적의 파라미터로 학습된 모델을 clf로 저장
clf = svc_param_selection(X_train, y_train.values.ravel(), 10)

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


## 👉최적의 비용 C : 1, 감마 : 0.1